In [ ]:
!pip install transformers tokenizers datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_filename = "naslovi_large.input"
dataset_filename = input_filename + '.prepared'
pretrained_model_name = 'distilgpt2'
new_model_name = pretrained_model_name + '_3'

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="/content/drive/MyDrive/ML-Vesti/naslovi-tokenizer.json")
tokenizer.pad_token = "<pad>"


In [ ]:
from datasets import load_dataset
# datasets = load_dataset('text', data_files=dataset_filename, split="train[90%],validate")
eval, train = load_dataset('text', data_files='/content/drive/MyDrive/ML-Vesti/' + dataset_filename, split=['train[:10%]', 'train[10%:]'])
def prepare_for_trainer(example):
  tokenized = tokenizer(example['text'], truncation=True, padding='max_length', max_length=20)
  tokenized["labels"] = tokenized["input_ids"].copy()
  return tokenized
tokenized_train = train.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])
tokenized_eval = eval.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])


Using custom data configuration default-0fd47dc85df26713


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-0fd47dc85df26713/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.
 

In [ ]:
from transformers import Trainer, AutoConfig, AutoModelForCausalLM, TrainingArguments

config = AutoConfig.from_pretrained(pretrained_model_name)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name)
# model.to('cuda:0')
training_args = TrainingArguments(
    new_model_name,
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    # weight_decay=0.01,
    save_strategy='epoch',
    num_train_epochs=10,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    output_dir='/content/drive/MyDrive/ML-Vesti/' + dataset_filename,
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    # prediction_loss_only=True,
    tokenizer=tokenizer,
)


loading configuration file https://huggingface.co/distilgpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a044e244dc69229682bebd7cacc81631
Model config GPT2Config {
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": tru

TypeError: ignored

In [ ]:
import logging
logger = logging.getLogger(__name__)
logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )

Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False


In [ ]:
train_result = trainer.train()
trainer.save_model('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)

***** Running training *****
  Num examples = 1127951
  Num Epochs = 10
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 44070


Epoch,Training Loss,Validation Loss
1,4.955600,5.726819
2,4.627600,5.361106
3,4.442600,5.148612
4,4.311200,4.993049
5,4.201200,4.882336
6,4.142400,4.806982
7,4.097800,4.756034
8,4.056800,4.720108
9,4.034800,4.699231
10,4.025200,4.693653


***** Running Evaluation *****
  Num examples = 125328
  Batch size = 256
Saving model checkpoint to distilgpt2_3/checkpoint-4407
Configuration saved in distilgpt2_3/checkpoint-4407/config.json
Model weights saved in distilgpt2_3/checkpoint-4407/pytorch_model.bin
tokenizer config file saved in distilgpt2_3/checkpoint-4407/tokenizer_config.json
Special tokens file saved in distilgpt2_3/checkpoint-4407/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 125328
  Batch size = 256
Saving model checkpoint to distilgpt2_3/checkpoint-8814
Configuration saved in distilgpt2_3/checkpoint-8814/config.json
Model weights saved in distilgpt2_3/checkpoint-8814/pytorch_model.bin
tokenizer config file saved in distilgpt2_3/checkpoint-8814/tokenizer_config.json
Special tokens file saved in distilgpt2_3/checkpoint-8814/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 125328
  Batch size = 256
Saving model checkpoint to distilgpt2_3/checkpoint-13221
Configuration

In [ ]:
import numpy as np
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)
model.config.pad_token_id = 1

for temp in np.linspace(0.2, 2, 10):
    print()
    print("###### Temperature: " + str(temp))
    for i in range(0, 5):
        out = model.generate(do_sample=True, temperature=temp)[0]
        print(tokenizer.decode(out, skip_special_tokens=True))

loading configuration file /content/drive/MyDrive/ML-Vesti/modeli/distilgpt2_3/config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.


###### Temperature: 0.2
U toku je broj zaraženih, a evo kako je policija uhapsila
Evo kako da se ponašate za život, a da li je to uradio!
U Beogradu se čekalo tri godine
U Srbiji je bio na svetu, a evo šta je sve potrebno je potrebno!
U toku je i dalje u Srbiji!

###### Temperature: 0.4
U Srbiji je bio na svetu, ali samo jedna stvarni!
U toku je velika operacija pomoć, a evo šta je potrebno je
U toku je jedan od najboljih stanja, a evo šta je rekao o čemu se radi
Evo kako su navijačičari navijačičari navijačičari navijačičari
Evo šta je sve potrebno, a šta je sve u Srbiji potrebno je

###### Temperature: 0.6000000000000001
Evo šta se dešava kada je policija traga virusom zemaljaskih službi Mitrovicejskog turnira u Čačku
U toku je jedan od najboljih glumaca opozicionara
U ovom gradu u Beogradunoj akciji Uzmi račun i pobedi redovima kuljenosti
Na današnji dan za pranje stolom, a evo kako da se zaštitite!
Evo šta je rekao o svom životunoj vezi sa suprugom devojkom (VIDEO)

###### Tempera